In [ ]:
# run for first time setup
! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

In [ ]:
import sys
sys.path.append("..\\")
import login
login.connect()
import datajoint as dj

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from ipywidgets.widgets import Layout, Label

In [ ]:
row_limit = 25
dj.config["display.limit"] = row_limit
# TODO:
# - implement navigation buttons to offset queries
# - foreign keys linked to lookup tables should give dropdown menus
# - print out current query
# - print out size on disk
# - support for part tables
import inspect
out = widgets.Output()

available_schema = dj.list_schemas()
schema_dropdown = widgets.Dropdown(options = available_schema, value = "common_mice")
dj_schema = dj.Schema("common_mice")
mod = dj.VirtualModule("temp.py", "common_mice", create_schema = False, create_tables=False)

def get_table_names(virtualmodule):
    names = []
    for i in inspect.getmembers(virtualmodule):
        a = getattr(virtualmodule, i[0])
        if type(a) == dj.user_tables.TableMeta:
            names.append(i[0])
    return names

tables = get_table_names(mod)
table_dropdown = widgets.Dropdown(options = tables)

@out.capture(clear_output=True, wait=True)
def schema_update(*args):
    mod = dj.VirtualModule("temp.py", schema_dropdown.value, create_schema = False, create_tables=False)
    tables = get_table_names(mod)
    table_dropdown.options = tables
schema_dropdown.observe(schema_update, "value")

def get_attribute_names(table_class):
    names = list(table_class().heading.attributes.keys())
    return names

@out.capture(clear_output=True, wait=True)
def show_table(restriction):
    mod = dj.VirtualModule("temp.py", schema_dropdown.value, create_schema = False, create_tables=False)
    table_class = getattr(mod, table_dropdown.value)
    table_attributes = get_attribute_names(table_class)
    if restriction == {}:
        display(table_class())
    else:
        display(table_class() & restriction)
    return table_class
#table_dropdown.observe(show_table, "value")
def get_lookup_table_attributes(table_class):
    indexes = []
    for i in table_class.heading.indexes:
        indexes.append(i[0])
    return indexes

def generate_attribute_widgets(table_class):
    w_attr = {}
    attributes = table_class.heading.attributes
    for i in attributes:
        if "enum" in attributes[i].type:
            enums = "[" + attributes[i].type.strip("enum()") + "]"
            options = [""] + eval(enums)
            w_attr[i] = widgets.Dropdown(options = options, value = "", description = i)
        if ("varchar" in attributes[i].type) or ("int" in attributes[i].type) or ("float" in attributes[i].type) or ("decimal" in attributes[i].type) or ("date" in attributes[i].type):
            w_attr[i] = widgets.Text(description = i)
    return w_attr

w_attr = {}
def restrict_from_widgets(*args):
    restriction = {}
    global w_attr
    for i in w_attr:
        if w_attr[i].value != "":
            restriction[i] = w_attr[i].value
    table_class = show_table(restriction)
    return restriction

update_button = widgets.Button(description="Update")
def update_display(*args):
    global vbox
    global w_attr
    vbox.close()
    w = {}
    w['schema'] = schema_dropdown
    w['table'] = table_dropdown
    w['button'] = update_button
    #print(controls.children)
    out.clear_output()
    label_layout = Layout(width='100px',height='30px')
    restrict_button = widgets.Button(description="Restrict")
    restrict_button.on_click(restrict_from_widgets)
    table_class = show_table({})
    w_attr = generate_attribute_widgets(table_class)
    controls1 = widgets.VBox([Label("Table selection")]+list(w.values()), layout = Layout(border='1px solid black'))
    controls2 = widgets.GridBox([Label("Attributes",layout=label_layout),restrict_button] + list(w_attr.values()), layout = Layout(border='1px solid black', grid_template_columns="repeat(2, 320px)"))
    controls3 = widgets.HBox([controls1, controls2])
    vbox = widgets.VBox([controls3, out])
    display(vbox)
update_button.on_click(update_display)

w = {}
w['schema'] = schema_dropdown
w['table'] = table_dropdown
w['button'] = update_button
controls = widgets.VBox(list(w.values()))
vbox = widgets.VBox([controls, out])
display(vbox)
update_display()